In [314]:
from BeautifulSoup import BeautifulSoup
import urllib2, sys
import re
import itertools
import pandas as pd

In [317]:
#code to request website info
url  = "http://www.menuism.com/restaurant-locations/waffle-house-78955/us/al"
hdr = {'User-Agent': 'Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.11 (KHTML, like Gecko) Chrome/23.0.1271.64 Safari/537.11',
       'Accept': 'text/html,application/xhtml+xml,application/xml;q=0.9,*/*;q=0.8',
       'Accept-Charset': 'ISO-8859-1,utf-8;q=0.7,*;q=0.3',
       'Accept-Encoding': 'none',
       'Accept-Language': 'en-US,en;q=0.8',
       'Connection': 'keep-alive'}

req = urllib2.Request(url, headers=hdr)

try:
    page = urllib2.urlopen(req)
except urllib2.HTTPError, e:
    print e.fp.read()

content = page.read()
soup = BeautifulSoup(content)

In [316]:
#code to grab all the addresses on the page
rawhtml = soup.findAll("ul", {"class":""})

#loop to create a list of addresses
itemList = []
for item in rawhtml:
    itemList.append(''.join(item.findAll(text=True)).split("\n\n\n"))
    print itemList

#flattening the list and removing white space
chain = itertools.chain(*itemList)
chainedList = (list(chain))
finalList = map(lambda x: x.strip(), chainedList)

#creating dataframe from list
data = pd.DataFrame()

[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]
[]


In [330]:
chain = itertools.chain(*itemList)
chainedList = (list(chain))
finalList = map(lambda x: x.strip(), chainedList)

dataframe = pd.DataFrame(finalList)
print dataframe

                                                     0
0         Waffle House in 36201 at 1201 S Quintard Ave
1           Waffle House in 36201 at 1530 Quintard Ave
2           Waffle House in 35954 at 802 Cleveland Ave
3     Waffle House in 35954 at 961 Gilbert Ferry Rd Se
4             Waffle House in 36830 at 110 W Glenn Ave
5          Waffle House in 36830 at 2346 Bent Creek Rd
6             Waffle House in 36830 at 1738 Opelika Rd
7           Waffle House in 36832 at 2167 S College St
8          Waffle House in 35022 at 3998 Methodist Cir
9             Waffle House in 35022 at 5010 Academy Ln
10              Waffle House in 35205 at 800 11th St S
11             Waffle House in 35206 at 9227 Parkway E
12              Waffle House in 35209 at 185 Oxmoor Rd
13   Waffle House in 35210 at 1833 Crestwood Boulevard
14        Waffle House in 35210 at 1833 Crestwood Blvd
15         Waffle House in 35216 at 3023 Columbiana Rd
16        Waffle House in 35216 at 1535 Montgomery Hwy
17    Waff